In [9]:
import json
import os
from dataload import ChessReD2KDataset
from torchvision import transforms

# 1. Carregar o ficheiro annotations.json
json_path = "data/annotations.json"
with open(json_path, 'r') as f:
    data = json.load(f)

# 2. Verificar as pastas de imagens disponíveis no disco (exemplo no Windows)
images_root = "data/images"
existing_folders = set(os.listdir(images_root))
print("Pastas de imagens existentes no disco:", existing_folders)

# 3. Filtrar imagens para só carregar as que tens no disco
filtered_images = [img for img in data['images'] if img['path'].split('/')[1] in existing_folders]

# 4. Obter os IDs das imagens filtradas
filtered_image_ids = set(img['id'] for img in filtered_images)

# 5. Filtrar anotações para só as imagens disponíveis
filtered_pieces = [piece for piece in data['annotations']['pieces'] if piece['image_id'] in filtered_image_ids]

# 6. Obter os IDs de imagens para treino/val/test dos splits originais
train_image_ids = set(data['splits']['train']['image_ids'])
val_image_ids = set(data['splits']['val']['image_ids'])
test_image_ids = set(data['splits']['test']['image_ids'])

# 7. Filtrar as imagens por split e existentes no disco
train_images = [img for img in filtered_images if img['id'] in train_image_ids]
val_images = [img for img in filtered_images if img['id'] in val_image_ids]
test_images = [img for img in filtered_images if img['id'] in test_image_ids]

print(f"Número de imagens treino filtradas: {len(train_images)}")
print(f"Número de imagens validação filtradas: {len(val_images)}")
print(f"Número de imagens teste filtradas: {len(test_images)}")

print("Info dataset:", data['info'])

# 8. Definir transformações
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

# 9. Definir o caminho base para as imagens (até à pasta 'data')
images_dir = "data"

# 10. Criar datasets PyTorch para treino, validação e teste
train_dataset = ChessReD2KDataset(images_dir, train_images, filtered_pieces, transform=transform)
val_dataset = ChessReD2KDataset(images_dir, val_images, filtered_pieces, transform=transform)
test_dataset = ChessReD2KDataset(images_dir, test_images, filtered_pieces, transform=transform)

# 11. Testar a leitura de uma imagem/anotação (opcional)
image, target = train_dataset[0]
print(image.shape)
print(target)


Pastas de imagens existentes no disco: {'22', '83', '33', '78', '47', '19', '0', '42', '28', '56', '91', '41', '38', '61', '58', '87', '72', '99', '76', '6'}
Número de imagens treino filtradas: 1442
Número de imagens validação filtradas: 330
Número de imagens teste filtradas: 306
Info dataset: {'description': 'Chess Recognition Dataset (ChessReD)', 'version': '1.0', 'year': 2023, 'date_created': '26/08/2023', 'author': 'Athanasios Masouris'}
torch.Size([3, 512, 512])
{'boxes': tensor([[2236.4399, 2033.0601,  171.6100,  211.0300],
        [1995.5100, 1994.8000,  156.9500,  245.8700],
        [1742.7100, 2013.5400,  153.2000,  214.1400],
        [1486.7900, 1941.2200,  175.5200,  280.1100],
        [1240.4301, 1921.5400,  175.3200,  293.2400],
        [ 997.8900, 1973.8400,  158.6300,  224.2100],
        [ 765.2600, 1952.1500,  161.7500,  234.4600],
        [ 501.2900, 1975.0500,  181.4700,  206.4000],
        [2230.0000, 1836.0800,  132.5100,  171.6100],
        [1996.6801, 1831.4399,  